## Setting

In [1]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn
import timm
from PIL import Image
from tqdm import tqdm
import random
import os

%config Completer.use_jedi = False

In [2]:
def setseed(seednum = 20):
    torch.manual_seed(seednum)
    torch.cuda.manual_seed(seednum)
    torch.cuda.manual_seed_all(seednum)
    np.random.seed(seednum)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seednum)

In [3]:
setseed(35)

In [4]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
device

device(type='cuda')

## Dataset

In [5]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [6]:
transform_aug1 = ts.Compose(
    [
        ts.ToTensor(),
        ts.RandomHorizontalFlip(p = 0.9),
        ts.RandomGrayscale(p=0.9),
        ts.RandomApply([ts.RandomRotation(180)], p=0.9)
    ]
)

In [7]:
transform_aug2 = ts.Compose(
    [
        ts.ToTensor(),
        ts.RandomApply([ts.RandomResizedCrop((32, 32), scale=(0.1, 1), ratio=(0.5, 2)),
        ts.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)], p=1)
    ]
)

In [8]:
train_set_origin = ds.CIFAR10(root='../data/', train=True, transform=transform, download=True)
train_set_aug1 = ds.CIFAR10(root='../data/', train=True, transform=transform_aug1, download=True)
train_set_aug2 = ds.CIFAR10(root='../data/', train=True, transform=transform_aug2, download=True)
val_set = tv.datasets.CIFAR10(root='../data/', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
train_set = train_set_origin + train_set_aug1 + train_set_aug2

In [10]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2)

In [11]:
def make_test_list(data_dir):
    
    test_img_list = list()
    
    files = os.listdir(data_dir)
    
    for i in range(10):
        img_file = data_dir + "/" + files[i]
        img = os.listdir(img_file)
        for j in range(200):
            img_path = data_dir + "/" +  files[i] + "/" + img[j]
            
            test_img_list.append(img_path)
    return test_img_list

In [12]:
img_list = make_test_list("../Statistical_Deep_Image_2021/Statistical_Deep_Image")

In [13]:
def make_test_label(label_list):
    
    test_label_list = list()
    
    for i in [0,2,1,3,4,5,6,7,8,9]:
        for j in range(200):
            test_label_list.append(i)
        
    return test_label_list

In [14]:
label = os.listdir("../Statistical_Deep_Image_2021/Statistical_Deep_Image")
img_label_list = make_test_label(label)

In [15]:
class testset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_label_list, transform):
        self.file_list = img_list
        self.labels = img_label_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = self.labels[index]
        return img_transformed, label

In [16]:
test_set = testset(img_list = img_list,
                  img_label_list = img_label_list,
                  transform = transform)

In [17]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle=False, num_workers=2)

In [18]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Model

In [19]:
class IdentityPadding(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityPadding, self).__init__()

        if stride == 2:
            self.pooling = nn.AvgPool2d(kernel_size=2, stride=2, ceil_mode=True)
        else:
            self.pooling = None
            
        self.add_channels = out_channels - in_channels
    
    def forward(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.add_channels))
        if self.pooling is not None:
            out = self.pooling(out)
        return out


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                                stride=stride, padding=1, bias=False)      
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, 
                                stride=1, padding=1, bias=False)    
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.down_sample = IdentityPadding(in_channels, out_channels, stride)
            
        self.stride = stride

    def forward(self, x):
        shortcut = self.down_sample(x)
        out = self.bn1(x)
        out = self.conv1(out)        
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn3(out)
       
        out += shortcut
        return out


class PyramidNet(nn.Module):
    def __init__(self, num_layers, alpha, block, num_classes=10):
        super(PyramidNet, self).__init__()   	
        self.in_channels = 16
        
        # num_layers = (110 - 2)/6 = 18
        self.num_layers = num_layers
        self.addrate = alpha / (3*self.num_layers*1.0)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, 
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        # feature map size = 32x32
        self.layer1 = self.get_layers(block, stride=1)
        # feature map size = 16x16
        self.layer2 = self.get_layers(block, stride=2)
        # feature map size = 8x8
        self.layer3 = self.get_layers(block, stride=2)

        self.out_channels = int(round(self.out_channels))
        self.bn_out= nn.BatchNorm2d(self.out_channels)
        self.relu_out = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.fc_out = nn.Linear(self.out_channels, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', 
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def get_layers(self, block, stride):
        layers_list = []
        for _ in range(self.num_layers - 1):
            self.out_channels = self.in_channels + self.addrate
            layers_list.append(block(int(round(self.in_channels)), 
                                     int(round(self.out_channels)), 
                                     stride))
            self.in_channels = self.out_channels
            stride=1

        return nn.Sequential(*layers_list)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.bn_out(x)
        x = self.relu_out(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_out(x)
        return x


def pyramidnet():
	block = ResidualBlock
	model = PyramidNet(num_layers=18, alpha=48, block=block)
	return model

In [20]:
model = pyramidnet()

In [21]:
model = model.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Train

In [26]:
def train(num_epoch):
    best_accuracy = 0.0 
    
    for epoch in tqdm(range(num_epoch)):

        running_train_loss = 0.0
        running_val_loss = 0.0
        true = 0
        total = 0

        for i, data in enumerate(train_loader, 0) :
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
        
        with torch.no_grad():
            model.eval()
            for i, data in enumerate(val_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                predicted_outputs = model(inputs)
                val_loss = criterion(predicted_outputs, labels)
                
                _, predicted = torch.max(predicted_outputs, 1) 
                
                running_val_loss += val_loss.item()
                total += labels.size(0)
                true += (predicted == labels).sum().item()
                
        train_loss_per_epoch = running_train_loss / len(train_loader)
        val_loss = running_val_loss/len(val_loader)
        accuracy = (100 * true / total)     
        
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), 'PyramidNet_Aug3_final_weights.pth')
            best_accuracy = accuracy
            
        print('epoch: %d' %(epoch+1), ' train_loss: %.3f' %train_loss_per_epoch, ' val_loss: %.4f' %val_loss, ' Accuracy: %.2f %%' % (accuracy))

In [27]:
train(40)

  2%|▎         | 1/40 [18:22<11:56:56, 1102.99s/it]

epoch: 1  train_loss: 1.723  val_loss: 0.9611  Accuracy: 66.65 %


  5%|▌         | 2/40 [35:48<11:17:03, 1069.03s/it]

epoch: 2  train_loss: 1.322  val_loss: 0.7086  Accuracy: 75.49 %


  8%|▊         | 3/40 [54:28<11:13:43, 1092.53s/it]

...

epoch: 38  train_loss: 0.437  val_loss: 0.3852  Accuracy: 91.64 %


 98%|█████████▊| 39/40 [11:47:18<18:33, 1113.07s/it]

epoch: 39  train_loss: 0.433  val_loss: 0.3709  Accuracy: 91.34 %


100%|██████████| 40/40 [12:05:55<00:00, 1088.88s/it]

epoch: 40  train_loss: 0.429  val_loss: 0.3533  Accuracy: 91.43 %


In [28]:
train(5)

 20%|██        | 1/5 [17:29<1:09:59, 1049.80s/it]

epoch: 1  train_loss: 0.427  val_loss: 0.3845  Accuracy: 91.86 %


 40%|████      | 2/5 [35:07<52:43, 1054.63s/it]  

epoch: 2  train_loss: 0.422  val_loss: 0.3658  Accuracy: 91.68 %


 60%|██████    | 3/5 [52:46<35:12, 1056.34s/it]

epoch: 3  train_loss: 0.419  val_loss: 0.3528  Accuracy: 91.74 %


 80%|████████  | 4/5 [1:10:42<17:44, 1064.42s/it]

epoch: 4  train_loss: 0.419  val_loss: 0.3757  Accuracy: 91.96 %


100%|██████████| 5/5 [1:29:03<00:00, 1068.64s/it]

epoch: 5  train_loss: 0.414  val_loss: 0.3494  Accuracy: 92.15 %


In [34]:
train(20)

  5%|▌         | 1/20 [18:50<5:57:54, 1130.22s/it]

epoch: 1  train_loss: 0.315  val_loss: 0.3483  Accuracy: 93.38 %


 10%|█         | 2/20 [37:45<5:39:58, 1133.27s/it]

epoch: 2  train_loss: 0.315  val_loss: 0.3534  Accuracy: 93.39 %


 10%|█         | 2/20 [46:14<6:56:10, 1387.25s/it]


KeyboardInterrupt: 

## Test

In [35]:
model.load_state_dict(torch.load('PyramidNet_Aug3_final_weights.pth'))
model.eval()

PyramidNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 17, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(17, 17, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (down_sample): IdentityPadding()
    )
    (1): ResidualBlock(
      (bn1): BatchNorm2d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(17, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2

In [36]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))

100%|██████████| 63/63 [00:05<00:00, 11.00it/s]

Accuracy: 74 %
